# Finding Collision for FNV2

In [86]:
import re

In [87]:
FNV_prime_32 =  2^24 + 2^8 + 0x93
FNV_prime_64 = 2^40 + 2^8 + 0xb3
FNV_prime_128 = 2^88 + 2^8 + 0x3b

offset_basis_32 = 2166136261
offset_basis_64 = 14695981039346656037
offset_basis_128 = 144066263297769815596495629667062367629

In [88]:
def fnv1a(hex_input, size, flag = 0):
    #Constants
    if size == 32:
        FNV_prime = FNV_prime_32
        offset_basis = offset_basis_32
        module = 2**32
    elif size == 64:
        FNV_prime = FNV_prime_64
        offset_basis = offset_basis_64
        module = 2**64
    elif size == 128:
        FNV_prime = FNV_prime_128
        offset_basis = offset_basis_128    
        module = 2**128    
    hash = offset_basis
    hex_input = hex_input.replace(" ", "")
    hex_input = re.findall('..', hex_input)
    for byte_of_data in hex_input:                
        if flag == 1:
            hash = mod(hash + int(byte_of_data, base=16), 2**128)
        else:
            hash = hash ^^ int(byte_of_data, base=16)
        hash = (hash * FNV_prime) % module
    return hash

In [89]:
fnv1a("8c2565b0f35411600c3c0e20e21235", 128)

314800192907571496543841540975179960854

In [90]:
def convert2hex(l):
    temp = list(map(lambda x: hex(x)[2:], l))
    output = temp
    for i in range(len(l)):
        if len(temp[i]) < 2:
            output[i] = temp[i].zfill(2)
    output = "".join(output)
    return output

# Using LLL to Find Collision

In [91]:
n = 31
t = 200
g = FNV_prime_128
E = []
temp = [0 for _ in range(n)] + [t * 2**128]
E.append(temp)
f = lambda i, j: (i == j)*1
for i in range(n):
    temp = [f(i, j) for j in range(n)] + [mod(g**(n - i), 2**128)]    
    E.append(temp)
A = matrix(ZZ, n + 1, n + 1, E)
B = A.LLL()
B1 = []
for i in range(B.nrows()):
    if B[i, -1] == 0:
        B1.append(B.row(i))

In [92]:
for selected_row in range(len(B1)):
    selected_coeffs = list(B1[selected_row])    
    tr = -1 * min(selected_coeffs)    
    X = [tr + selected_coeffs[i] for i in range(n)]    
    Y = [tr for _ in range(n)] 
    X = convert2hex(X)
    Y = convert2hex(Y)
    if fnv1a(X, 128, 1) == fnv1a(Y, 128, 1):
        print("%s = %s" % ("X", X))
        print("%s = %s" % ("Y", Y))
        print("%s = %s" % ("fnv1a(X)", (fnv1a(X, 128, 1))))
        print("%s = %s" % ("fnv1a(Y)", (fnv1a(Y, 128, 1))))

X = 11140a12140b110b0b0e0d02060f0904170b140600161609080f0b090d0d0d
Y = 0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d
fnv1a(X) = 245130900151624955695674862832482876426
fnv1a(Y) = 245130900151624955695674862832482876426
X = 1808111616000c0807070a06030b06110a1514040d11140e0b04030c0d0710
Y = 0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d0d
fnv1a(X) = 245130900151624955695674862832482876426
fnv1a(Y) = 245130900151624955695674862832482876426
X = 0a16030c000f090d0307131c150e100d0908110c130c130c050d070b0d0e13
Y = 0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b
fnv1a(X) = 90194142264012191722037330296100703884
fnv1a(Y) = 90194142264012191722037330296100703884


# The End